In [ ]:
library(DESeq2)
library(DEGreport)
library(tidyverse)
library(patchwork)

In [ ]:
readcount <- read.table('./source_data/batch_correction/batch_cor_TECount_filled_mfa.txt',header=T)

split_result = str_split(readcount$TE_ID,"\\|", 6)
readcount$TE = lapply(X = split_result, FUN = "[", 4)
rownames(readcount) <- readcount$TE_ID


split_result_sub = str_split(readcount$TE,"\\:", 3)
readcount$TEs = lapply(X = split_result_sub, FUN = "[", 1)
readcount <- readcount %>% group_by(TEs) %>% summarise(across(c(A_FST,B_FST,A_SND,B_SND,A_TRD,B_TRD,
                                                                C_FST,D_FST,C_SND,D_SND,C_TRD,D_TRD,
                                                                E_FST,F_FST,E_SND,F_SND,E_TRD,F_TRD,
                                                                G_FST,H_FST,G_SND,H_SND,G_TRD,H_TRD),sum),
                                                                    .groups = 'drop') %>% as.data.frame()

gene_symbol <- readcount$TEs
rownames(readcount) <- readcount$TEs
readcount <- readcount[,c(2:25)]
readcount[, c(1:24)] <- sapply(readcount[, c(1:24)], as.numeric)
readcount <- round(readcount)

In [ ]:
expdesign <- read.table("./source_data/reference/mfa_samplelist.txt",row.names = 1)
expdesign <- expdesign[c(1,2,9,10,17,18,3,4,11,12,19,20,5,6,13,14,21,22,7,8,15,16,23,24),]
expdesign$condition = factor(c(rep('G1',6),rep('G2',6),rep('G3',6),rep('G4',6)))

In [ ]:
##############
# LRT method #
##############

dds <- DESeqDataSetFromMatrix(countData = readcount,
                              colData = expdesign,
                              design = ~ condition)

dds_LRT <- DESeq(dds, test="LRT", reduced = ~1)
res_LRT <-results(dds_LRT)


count <- counts(dds_LRT, normalized = TRUE)
design <- as.data.frame(colData(dds_LRT))

basemean <- assay(rlog(dds_LRT, blind=FALSE))

In [ ]:
write.table(count, file='./source_data/DETE/batch_cor_TE_sum_LRT_normalized_count_TC.txt', sep='\t',col.names = T, row.names = T)

In [ ]:
################################################
#detect patterns of expression & pattern plot
################################################

sig_res_LRT <- res_LRT %>%
  data.frame() %>%
  rownames_to_column(var="gene") %>% 
  as_tibble() %>% 
  filter(padj < 0.001)

sig_res_LRT <- sig_res_LRT %>%
  arrange(padj)

sigLRT_genes <- sig_res_LRT %>% 
  pull(gene)

cluster_rlog <- basemean[sigLRT_genes, ]

In [ ]:
################
# pattern plot #
################
clusters <- degPatterns(cluster_rlog, metadata = expdesign, time = "condition", minc =15 ,reduce=TRUE)

## Supplementary Fig. 8b 

In [ ]:
################
# pattern plot #
################

clusters_patterns <- clusters
options(repr.plot.width = 40,repr.plot.height =10)

for(i in unique(clusters_patterns$normalized$cluster)){    
    clusters_group <- clusters_patterns$normalized[clusters_patterns$normalized$cluster == i,]
    N <- nrow(clusters_group)/4
    Plt = ggplot(clusters_group, aes(x = condition, y = value)) + theme_bw()+ coord_fixed(ratio = 0.9) +
    theme(panel.grid.major = element_blank(),panel.grid.major.x = element_line(),panel.grid.major.y = element_line())+
    geom_boxplot(aes(color=condition),alpha = 0.5,width = 0.3, outlier.shape = NA, size = 1)+
    geom_point(aes(color=condition),shape = 20, size =5, alpha = 0.7 ,position = position_jitter(0.05))+
    geom_line(aes(group=genes,color=condition),size = 0.1, alpha = 0.1)+
    geom_text(x=1.2, y=2, label=paste0("n=",N),size=15)+
    theme(axis.text.y= element_text(lineheight = 1, size = 50),
          axis.text.x= element_text(size = 50),
          axis.line.y = element_line(size = 1),
          axis.line.x = element_line(size = 1),
          axis.title = element_blank(),
          panel.border = element_blank(),
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          axis.line = element_line(colour = "black"),
          plot.title = element_text(size = 50))+
    theme(legend.position="none") + ggtitle(paste0("Pattern",i)) + ggeasy::easy_center_title()+
    scale_y_continuous(breaks=c(-1.5,0,1.5),limits=c(-2, 2))
    
    print(Plt)
}

In [ ]:
################
# parsing data #
################
cluster_groups <- clusters$df

data <- cbind(res_LRT,basemean)
data <- as.data.frame(data)

data1 <- merge(cluster_groups,data,by.x = 0, by.y = 0)
names(data1)[2] <- c("gene")

write.table(data1[data1$cluster=="1",][,c(5,9,2,1)],file='./source_data/DETE/TEsum_group1_padj_score.txt',sep='\t',col.names = T,row.names = F)
write.table(data1[data1$cluster=="2",][,c(5,9,2,1)],file='./source_data/DETE/TEsum_group2_padj_score.txt',sep='\t',col.names = T,row.names = F)